## 准备数据

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [4]:
# class myModel:
#     def __init__(self):
#         ####################
#         '''声明模型对应的参数'''
#         ####################
#     def __call__(self, x):
#         ####################
#         '''实现模型函数体，返回未归一化的logits'''
#         ####################
#         return logits
        


class myModel:
    def __init__(self):
        # 声明模型对应的参数
        self.W1 = tf.Variable(tf.random.truncated_normal([784, 256], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([256]))
        self.W2 = tf.Variable(tf.random.truncated_normal([256, 10], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([10]))
        
    def __call__(self, x):
        x = tf.reshape(x, [-1, 784])  # Flatten the input
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)  # First layer
        logits = tf.matmul(h1, self.W2) + self.b2  # Output layer
        return logits


model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [5]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [6]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.6634269 ; accuracy 0.08513334
epoch 1 : loss 2.6270988 ; accuracy 0.08636667
epoch 2 : loss 2.5933359 ; accuracy 0.08731667
epoch 3 : loss 2.5617328 ; accuracy 0.088083334
epoch 4 : loss 2.5319865 ; accuracy 0.0895
epoch 5 : loss 2.5038588 ; accuracy 0.09091666
epoch 6 : loss 2.4771662 ; accuracy 0.09278333
epoch 7 : loss 2.4517539 ; accuracy 0.09561667
epoch 8 : loss 2.4274955 ; accuracy 0.09821667
epoch 9 : loss 2.404289 ; accuracy 0.101166666
epoch 10 : loss 2.3820424 ; accuracy 0.10455
epoch 11 : loss 2.3606741 ; accuracy 0.10893334
epoch 12 : loss 2.3401113 ; accuracy 0.11366667
epoch 13 : loss 2.3202896 ; accuracy 0.11841667
epoch 14 : loss 2.301153 ; accuracy 0.123966664
epoch 15 : loss 2.2826445 ; accuracy 0.1301
epoch 16 : loss 2.2647178 ; accuracy 0.13733333
epoch 17 : loss 2.247329 ; accuracy 0.1454
epoch 18 : loss 2.2304344 ; accuracy 0.15353334
epoch 19 : loss 2.2139978 ; accuracy 0.1618
epoch 20 : loss 2.1979823 ; accuracy 0.17053333
epoch 21 : loss 2.182